In [1]:
import torch
from gcn import GCN
from utils import load_data, preprocess, normalize_adj_tensor, accuracy, get_train_val_test
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from metattack import MetaApprox, Metattack
import seaborn as sns
import gc

In [3]:
# Fixed parameters for Cora dataset
seed = 15
epochs = 200
lr = 0.01
hidden = 16
dataset = 'cora'
ptb_rate = 0.05
model_variant = 'Meta-Self'  # Options: 'A-Meta-Self', 'Meta-Self'

In [4]:
# Set seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != 'cpu':
    torch.cuda.manual_seed(seed)

# Load Cora dataset
adj, features, labels = load_data(dataset=dataset)
nclass = max(labels) + 1

Loading cora dataset...
reading cora...
Selecting 1 largest connected components


In [5]:
# Split dataset
val_size = 0.1
test_size = 0.8
train_size = 1 - test_size - val_size

idx = np.arange(adj.shape[0])
idx_train, idx_val, idx_test = get_train_val_test(idx, train_size, val_size, test_size, stratify=labels)
idx_unlabeled = np.union1d(idx_val, idx_test)
perturbations = int(ptb_rate * (adj.sum()//2))

# Preprocess without normalizing adjacency yet
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)

In [6]:
# Move data to device
if device != 'cpu':
    adj = adj.to(device)
    features = features.to(device)
    labels = labels.to(device)

In [7]:
def train_gcn(adj):
    ''' Train GCN on the given adjacency matrix '''
    # Normalize adjacency matrix for GCN
    norm_adj = normalize_adj_tensor(adj)
    
    # Initialize GCN model
    gcn = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=labels.max().item() + 1,
              dropout=0.5)
    
    if device != 'cpu':
        gcn = gcn.to(device)
    
    optimizer = optim.Adam(gcn.parameters(),
                           lr=lr, weight_decay=5e-4)
    
    # Train GCN
    gcn.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = gcn(features, norm_adj)
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        loss_train.backward()
        optimizer.step()
    
    return gcn


In [8]:
def evaluate(gcn, adj, idx):
    ''' Evaluate GCN on the given adjacency matrix and index '''
    # Normalize adjacency matrix for evaluation
    norm_adj = normalize_adj_tensor(adj)
    
    gcn.eval()
    with torch.no_grad():
        output = gcn(features, norm_adj)
        loss = F.nll_loss(output[idx], labels[idx])
        acc = accuracy(output[idx], labels[idx])
    
    return acc.item()

In [11]:
def run_attack():

    
    print('=== Training GCN on original(clean) graph ===')
    # First, train the GCN on the clean graph
    trained_gcn = train_gcn(adj)
    
    print('=== Setting up attack model ===')
    # Set up attack model parameters
    if 'Self' in model_variant:
        lambda_ = 0
    if 'Train' in model_variant:
        lambda_ = 1
    if 'Both' in model_variant:
        lambda_ = 0.5
    if 'A' in model_variant:
        model = MetaApprox(nfeat=features.shape[1], hidden_sizes=[hidden],
                        nnodes=adj.shape[0], nclass=nclass, dropout=0.5,
                        train_iters=0, attack_features=False, lambda_=lambda_, device=device)
    else:
        model = Metattack(nfeat=features.shape[1], hidden_sizes=[hidden],
                        nnodes=adj.shape[0], nclass=nclass, dropout=0.5,
                        train_iters=0, attack_features=False, lambda_=lambda_, device=device)
    
    if device != 'cpu':
        model = model.to(device)
        
    # Generate adversarial adjacency matrix focusing on test nodes
    print('=== Perturbing graph for evasion attack ===')
    
    modified_adj = model(features, adj, labels, idx_train, idx_test, perturbations, ll_constraint=False)
    modified_adj = modified_adj.detach()
    
    
    
    
    runs = 3  # Reduced from 10 to 3 for quicker execution in kernel
    clean_acc = []
    attacked_acc = []
    
    print('=== Evaluating GCN performance ===')
    # Test the already trained GCN on both clean and perturbed graph
    for i in range(runs):
        # Reset the GCN for each run
        trained_gcn = train_gcn(adj)
        
        # Evaluate on clean test data
        clean_acc.append(evaluate(trained_gcn, adj, idx_test))
        
        # Evaluate on perturbed test data (evasion)
        attacked_acc.append(evaluate(trained_gcn, modified_adj, idx_test))
        
        print(f"Run {i+1}/{runs}: Clean acc = {clean_acc[-1]:.4f}, Attacked acc = {attacked_acc[-1]:.4f}")
    
    # Calculate effectiveness metrics
    clean_mean = np.mean(clean_acc)
    clean_std = np.std(clean_acc)
    attack_mean = np.mean(attacked_acc)
    attack_std = np.std(attacked_acc)
    acc_drop = clean_mean - attack_mean
    relative_drop = (acc_drop / clean_mean) * 100
    
    # Print summary statistics without plotting
    print("\n=== Attack Effectiveness Summary ===")
    print(f"Clean accuracy: {clean_mean:.4f} ± {clean_std:.4f}")
    print(f"Attacked accuracy: {attack_mean:.4f} ± {attack_std:.4f}")
    print(f"Absolute accuracy drop: {acc_drop:.4f}")
    print(f"Relative accuracy drop: {relative_drop:.2f}%")
    print(f"Perturbation rate: {ptb_rate*100:.2f}% of edges")
    print(f"Effectiveness ratio: {acc_drop/ptb_rate:.4f} (drop per perturbation unit)")
    
    # Assessment
    if acc_drop > 0.10:
        print("Attack assessment: Highly effective")
    elif acc_drop > 0.05:
        print("Attack assessment: Moderately effective")
    elif acc_drop > 0.02:
        print("Attack assessment: Slightly effective")
    else:
        print("Attack assessment: Minimally effective")
    
    return {
        'clean_adj': adj,
        'modified_adj': modified_adj,
        'clean_acc': clean_acc,
        'attacked_acc': attacked_acc,
        'accuracy_drop': acc_drop,
        'relative_drop': relative_drop,
        'effectiveness_ratio': acc_drop/ptb_rate
    }

In [12]:
if __name__ == '__main__':
    torch.cuda.empty_cache()
    results = run_attack()

=== Training GCN on original(clean) graph ===
=== Setting up attack model ===
=== Perturbing graph for evasion attack ===
=== training surrogate model to predict unlabled data for self-training


Perturbing graph:   3%|█▊                                                               | 7/253 [00:00<00:07, 31.84it/s]

GCN loss on unlabled data: 1.9424426555633545
GCN acc on unlabled data: 0.1529175050301811
attack loss: 1.938445806503296
GCN loss on unlabled data: 1.9954707622528076
GCN acc on unlabled data: 0.11921529175050302
attack loss: 1.993355393409729
GCN loss on unlabled data: 1.9460983276367188
GCN acc on unlabled data: 0.1579476861167002
attack loss: 1.9494891166687012
GCN loss on unlabled data: 1.9732766151428223
GCN acc on unlabled data: 0.1433601609657948
attack loss: 1.9742162227630615
GCN loss on unlabled data: 1.9716851711273193
GCN acc on unlabled data: 0.15392354124748492
attack loss: 1.9786776304244995
GCN loss on unlabled data: 2.0223653316497803
GCN acc on unlabled data: 0.10261569416498995
attack loss: 2.020887613296509
GCN loss on unlabled data: 1.951534628868103
GCN acc on unlabled data: 0.17002012072434608
attack loss: 1.9435147047042847


Perturbing graph:   6%|███▊                                                            | 15/253 [00:00<00:06, 34.82it/s]

GCN loss on unlabled data: 1.964990258216858
GCN acc on unlabled data: 0.1267605633802817
attack loss: 1.967976689338684
GCN loss on unlabled data: 1.9173896312713623
GCN acc on unlabled data: 0.25201207243460766
attack loss: 1.9163750410079956
GCN loss on unlabled data: 1.9755786657333374
GCN acc on unlabled data: 0.10110663983903422
attack loss: 1.9717317819595337
GCN loss on unlabled data: 2.0282528400421143
GCN acc on unlabled data: 0.1051307847082495
attack loss: 2.0330450534820557
GCN loss on unlabled data: 2.018251419067383
GCN acc on unlabled data: 0.11468812877263583
attack loss: 2.0134027004241943
GCN loss on unlabled data: 1.9227910041809082
GCN acc on unlabled data: 0.16851106639839036
attack loss: 1.9240869283676147
GCN loss on unlabled data: 1.9946948289871216
GCN acc on unlabled data: 0.0875251509054326
attack loss: 1.9988640546798706
GCN loss on unlabled data: 2.0041208267211914
GCN acc on unlabled data: 0.130784708249497
attack loss: 2.0121963024139404


...

GCN loss on unlabled data: 1.9787498712539673
GCN acc on unlabled data: 0.12525150905432597
attack loss: 1.975260853767395
GCN loss on unlabled data: 1.974736213684082
GCN acc on unlabled data: 0.10462776659959759
attack loss: 1.9715099334716797
GCN loss on unlabled data: 2.000192642211914
GCN acc on unlabled data: 0.1695171026156942
attack loss: 1.9929574728012085
GCN loss on unlabled data: 1.9362059831619263
GCN acc on unlabled data: 0.1579476861167002
attack loss: 1.9320331811904907
GCN loss on unlabled data: 2.042463541030884
GCN acc on unlabled data: 0.13329979879275655
attack loss: 2.0278778076171875
GCN loss on unlabled data: 2.0394742488861084
GCN acc on unlabled data: 0.14839034205231388
attack loss: 2.0293078422546387
GCN loss on unlabled data: 2.016479253768921
GCN acc on unlabled data: 0.11519114688128773
attack loss: 2.0204854011535645
GCN loss on unlabled data: 1.9687978029251099
GCN acc on unlabled data: 0.12273641851106641
attack loss: 1.9627153873443604


Perturbing graph:  97%|█████████████████████████████████████████████████████████████▎ | 246/253 [00:06<00:00, 39.70it/s]

GCN loss on unlabled data: 1.9654208421707153
GCN acc on unlabled data: 0.1388329979879276
attack loss: 1.963767170906067
GCN loss on unlabled data: 1.9767869710922241
GCN acc on unlabled data: 0.11016096579476863
attack loss: 1.9709746837615967
GCN loss on unlabled data: 1.9062830209732056
GCN acc on unlabled data: 0.22082494969818917
attack loss: 1.8989132642745972
GCN loss on unlabled data: 1.9855810403823853
GCN acc on unlabled data: 0.1262575452716298
attack loss: 1.9799253940582275
GCN loss on unlabled data: 1.9758323431015015
GCN acc on unlabled data: 0.12173038229376258
attack loss: 1.9715839624404907
GCN loss on unlabled data: 1.9792414903640747
GCN acc on unlabled data: 0.10412474849094568
attack loss: 1.984615683555603
GCN loss on unlabled data: 1.9923369884490967
GCN acc on unlabled data: 0.1006036217303823
attack loss: 1.9897130727767944
GCN loss on unlabled data: 1.9636690616607666
GCN acc on unlabled data: 0.15543259557344066
attack loss: 1.96160089969635


Perturbing graph: 100%|███████████████████████████████████████████████████████████████| 253/253 [00:06<00:00, 39.54it/s]


GCN loss on unlabled data: 1.9515295028686523
GCN acc on unlabled data: 0.13229376257545272
attack loss: 1.9533600807189941
GCN loss on unlabled data: 2.016955852508545
GCN acc on unlabled data: 0.09507042253521128
attack loss: 2.0216972827911377
GCN loss on unlabled data: 1.9484362602233887
GCN acc on unlabled data: 0.14738430583501008
attack loss: 1.9518476724624634
GCN loss on unlabled data: 1.9677075147628784
GCN acc on unlabled data: 0.16096579476861167
attack loss: 1.9651166200637817
=== Evaluating GCN performance ===
Run 1/3: Clean acc = 0.8305, Attacked acc = 0.7998
Run 2/3: Clean acc = 0.8249, Attacked acc = 0.7918
Run 3/3: Clean acc = 0.8270, Attacked acc = 0.8033

=== Attack Effectiveness Summary ===
Clean accuracy: 0.8275 ± 0.0023
Attacked accuracy: 0.7983 ± 0.0048
Absolute accuracy drop: 0.0292
Relative accuracy drop: 3.53%
Perturbation rate: 5.00% of edges
Effectiveness ratio: 0.5835 (drop per perturbation unit)
Attack assessment: Slightly effective
